In [1]:
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
import xgboost as xgb
from operator import itemgetter

In [3]:
df = pd.read_csv('../input/train_folds_updated.csv')
df.shape

(1456, 43)

In [4]:
df['week_start_date'].describe()

count           1456
unique          1049
top       2004-01-08
freq               2
Name: week_start_date, dtype: object

In [5]:
df['week_start_date'].min(), df['week_start_date'].max()

('1990-04-30', '2010-06-25')

In [2]:
df = pd.read_csv('../input/train_folds_updated.csv')
X = df.drop(['total_cases', 'week_start_date', 'city'], axis=1)
X = add_constant(X)

vif_info = pd.DataFrame()
vif_info['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif_info['Columns'] = X.columns
vif_info.sort_values('VIF', ascending=False)

C:\Users\lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\stats\outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\regression\linear_model.py:1736: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


,VIF,Columns
6,inf,precipitation_amt_mm
14,inf,reanalysis_sat_precip_amt_mm
9,589.422977,reanalysis_dew_point_temp_k
13,400.018001,reanalysis_relative_humidity_percent
15,381.423516,reanalysis_specific_humidity_g_per_kg
7,368.852683,reanalysis_air_temp_k
23,160.173046,month
25,150.795475,dayofyear
8,71.543282,reanalysis_avg_temp_k
30,71.489181,month_sin


In [3]:
X.drop('reanalysis_dew_point_temp_k', axis=1, inplace=True)

In [5]:
X.drop('reanalysis_specific_humidity_g_per_kg', axis=1, inplace=True)

In [7]:
X.drop('month', axis=1, inplace=True)

In [9]:
X.drop('reanalysis_air_temp_k', axis=1, inplace=True)

In [11]:
X.drop('month_sin', axis=1, inplace=True)

In [13]:
X.drop('month_cos', axis=1, inplace=True)

In [15]:
X.drop('reanalysis_tdtr_k', axis=1, inplace=True)

In [17]:
X.drop('dayofweek', axis=1, inplace=True)

In [19]:
X.drop('dayofyear', axis=1, inplace=True)

In [21]:
X.drop('reanalysis_max_air_temp_k', axis=1, inplace=True)

In [23]:
X.drop('dayofyear_cos', axis=1, inplace=True)

In [25]:
X.drop('dayofyear_sin', axis=1, inplace=True)

In [27]:
X.drop('quarter', axis=1, inplace=True)

In [29]:
X.drop('station_avg_temp_c', axis=1, inplace=True)

In [31]:
X.drop('station_diur_temp_rng_c', axis=1, inplace=True)

In [33]:
X.drop('season', axis=1, inplace=True)

In [35]:
X.drop('reanalysis_min_air_temp_k', axis=1, inplace=True)

In [37]:
X.drop('is_weekday', axis=1, inplace=True)

In [66]:
vif_info = pd.DataFrame()
vif_info['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif_info['Columns'] = X.columns
vif_info.sort_values('VIF', ascending=False)
vif_info.to_csv('vif_frame.csv', index=False)

C:\Users\lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\stats\outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\regression\linear_model.py:1736: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


In [68]:
df = pd.read_csv('vif_frame.csv')
X = df.drop(['total_cases', 'week_start_date', 'city', ''], axis=1)
y = df['total_cases']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
model = DecisionTreeRegressor()
rfe = RFE(estimator=model, n_features_to_select=0.5)
rfe.fit(X_train, y_train)

features = X_train.columns.to_list()
for x, y in (sorted(zip(rfe.ranking_, features), key=itemgetter(0))):
    print(x, y)

KeyError: "['total_cases', 'week_start_date', 'city', 'kfold'] not found in axis"

In [69]:
df = pd.read_csv('train_folds_updated.csv')
X = df.drop(['total_cases', 'week_start_date', 'city', 'kfold'], axis=1)
y = df['total_cases']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
model = DecisionTreeRegressor()
rfe = RFE(estimator=model, n_features_to_select=0.5)
rfe.fit(X_train, y_train)

features = X_train.columns.to_list()
for x, y in (sorted(zip(rfe.ranking_, features), key=itemgetter(0))):
    print(x, y)

1 year
1 weekofyear
1 ndvi_ne
1 ndvi_nw
1 ndvi_se
1 ndvi_sw
1 reanalysis_air_temp_k
1 reanalysis_avg_temp_k
1 reanalysis_dew_point_temp_k
1 reanalysis_max_air_temp_k
1 reanalysis_min_air_temp_k
1 reanalysis_precip_amt_kg_per_m2
1 reanalysis_tdtr_k
1 station_avg_temp_c
1 station_diur_temp_rng_c
1 station_min_temp_c
1 station_precip_mm
1 dayofweek
1 dayofyear
1 month_sin
1 month_cos
1 dayofweek_sin
1 dayofweek_cos
1 dayofyear_sin
1 dayofyear_cos
1 quarter_sin
1 season_cos
2 reanalysis_specific_humidity_g_per_kg
3 precipitation_amt_mm
4 reanalysis_relative_humidity_percent
5 station_max_temp_c
6 month
7 reanalysis_sat_precip_amt_mm
8 is_weekend
9 is_weekday
10 season
11 quarter_cos
12 quarter
13 season_sin
14 abs_energy
15 count_above_mean
16 count_below_mean
17 mean_abs_change
18 mean_change
19 sum_values
20 sum_of_reoccurring_values
21 sum_of_reoccurring_data_points
22 sample_entropy
23 root_mean_square
24 ratio_value_number_to_time_series_length
25 ratio_beyond_r_sigma
26 range_count
2

In [45]:
df = pd.read_csv('../input/train_folds_updated.csv')
X = df.drop(['total_cases', 'week_start_date', 'city', 'kfold'], axis=1)
y = df['total_cases']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
model = xgb.XGBRegressor()
rfe = RFE(estimator=model, n_features_to_select=0.5)
rfe.fit(X_train, y_train)

features = X_train.columns.to_list()
for x, y in (sorted(zip(rfe.ranking_, features), key=itemgetter(0))):
    print(x, y)

1 year
1 weekofyear
1 ndvi_nw
1 ndvi_se
1 reanalysis_air_temp_k
1 reanalysis_dew_point_temp_k
1 reanalysis_max_air_temp_k
1 reanalysis_min_air_temp_k
1 reanalysis_precip_amt_kg_per_m2
1 reanalysis_relative_humidity_percent
1 station_avg_temp_c
1 station_diur_temp_rng_c
1 station_max_temp_c
1 station_min_temp_c
1 station_precip_mm
1 dayofweek
1 dayofyear
1 season
1 month_sin
1 month_cos
1 dayofweek_sin
1 dayofweek_cos
1 dayofyear_sin
1 dayofyear_cos
1 quarter_sin
1 quarter_cos
1 season_cos
2 ndvi_sw
3 month
4 precipitation_amt_mm
5 reanalysis_tdtr_k
6 ndvi_ne
7 reanalysis_specific_humidity_g_per_kg
8 reanalysis_avg_temp_k
9 season_sin
10 reanalysis_sat_precip_amt_mm
11 is_weekday
12 abs_energy
13 count_above_mean
14 count_below_mean
15 mean_abs_change
16 mean_change
17 sum_values
18 sum_of_reoccurring_values
19 sum_of_reoccurring_data_points
20 sample_entropy
21 root_mean_square
22 ratio_value_number_to_time_series_length
23 ratio_beyond_r_sigma
24 range_count
25 25thquantile
26 quarter

In [ ]:
year
weekofyear
ndvi_nw
ndvi_se
reanalysis_air_temp_k
reanalysis_dew_point_temp_k
reanalysis_max_air_temp_k
reanalysis_min_air_temp_k
reanalysis_precip_amt_kg_per_m2
reanalysis_relative_humidity_percent
1 station_avg_temp_c
1 station_diur_temp_rng_c
1 station_max_temp_c
1 station_min_temp_c
1 station_precip_mm
1 dayofweek
1 dayofyear
1 season
1 month_sin
1 month_cos
1 dayofweek_sin
1 dayofweek_cos
1 dayofyear_sin
1 dayofyear_cos
1 quarter_sin
1 quarter_cos
1 season_cos

In [77]:
feat = pd.read_csv('features.txt', sep=" ", header=None)
feat.columns = ['ID', 'Column Name']
feat

,ID,Column Name
0,1,year
1,1,weekofyear
2,1,ndvi_ne
3,1,ndvi_nw
4,1,ndvi_se
5,1,ndvi_sw
6,1,reanalysis_air_temp_k
7,1,reanalysis_avg_temp_k
8,1,reanalysis_dew_point_temp_k
9,1,reanalysis_max_air_temp_k


In [81]:
feat_to_c = feat['Column Name']
feat_list = feat_to_c.tolist()

In [61]:
df = pd.read_csv('../input/train_folds_updated.csv')
X = df.drop(['total_cases', 'week_start_date', 'city', 'kfold'], axis=1)
y = df['total_cases']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
model = xgb.XGBRegressor()
model.fit(X_train, y_train)

feat_imp = model.get_booster().get_score(importance_type="gain")
feat_imp_df = pd.DataFrame(feat_imp, index=[0])
feat_imp_df.to_csv('feature_importance_xgboost_updated.csv', index=False)

In [62]:
feat_imp = pd.read_csv('feature_importance_xgboost_updated.csv')
feat_imp = feat_imp.T
feat_imp = feat_imp.reset_index()
feat_imp.columns = ['Column Name', 'Column Value']
feat_imp.sort_values(ascending=False, by='Column Value')

,Column Name,Column Value
31,quarter_sin,14784.661133
4,ndvi_se,8024.532227
34,season_cos,3884.493408
27,dayofweek_sin,3140.338623
0,year,1974.721558
23,dayofyear,1711.198730
28,dayofweek_cos,1404.229980
1,weekofyear,1125.413940
7,reanalysis_air_temp_k,1072.044556
22,dayofweek,886.241760


In [59]:
feat_imp = pd.read_csv('feature_importance_xgboost.csv')
feat_imp = feat_imp.T
feat_imp = feat_imp.reset_index()
feat_imp.columns = ['Column Name', 'Column Value']
feat_imp.sort_values(ascending=False, by='Column Value')

,Column Name,Column Value
4,ndvi_se,21241.923828
1,weekofyear,7116.315430
0,year,5264.227539
28,dayofweek_cos,5032.597168
25,month_sin,4030.787842
27,dayofweek_sin,3953.099609
7,reanalysis_air_temp_k,3278.393311
30,dayofyear_cos,1692.452881
9,reanalysis_dew_point_temp_k,1200.925049
22,dayofweek,1176.527954


In [63]:
from sklearn.feature_selection import SelectFromModel

In [65]:
df = pd.read_csv('../input/train_folds_updated.csv')
X = df.drop(['total_cases', 'week_start_date', 'city', 'kfold'], axis=1)
col_names = X.columns
y = df['total_cases']

model = xgb.XGBRegressor()

sfm = SelectFromModel(estimator=model)
X_transformed = sfm.fit_transform(X, y)

support = sfm.get_support()

print([
x for x, y in zip(col_names, support) if y == True
])

['year', 'weekofyear', 'ndvi_se', 'dayofweek', 'season', 'month_sin', 'dayofweek_sin', 'dayofweek_cos', 'quarter_sin', 'season_cos']


In [3]:
df = pd.read_csv('train_folds.csv')
df.head()

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases,kfold
0,sj,2004,2,2004-01-08,-0.07310,0.01720,0.214729,0.222443,0.00,297.648571,...,0.00,13.881429,2.028571,24.385714,5.557143,27.8,20.6,15.6,20,0
1,sj,2000,37,2000-09-16,0.05770,0.15070,0.297186,0.254343,30.52,300.794286,...,30.52,18.314286,2.285714,28.185714,6.985714,33.3,22.8,9.3,6,0
2,sj,1992,25,1992-06-17,0.10445,0.15315,0.181157,0.192086,52.07,298.931429,...,52.07,17.441429,2.000000,28.285714,6.742857,32.8,24.4,29.8,26,0
3,sj,1991,40,1991-10-01,0.12910,0.16672,0.177000,0.195557,31.68,299.650000,...,31.68,17.850000,2.042857,27.742857,6.914286,32.8,23.3,26.5,140,0
4,sj,1992,34,1992-08-19,0.11460,0.13680,0.185586,0.192571,68.22,299.491429,...,68.22,17.601429,2.242857,28.214286,6.928571,33.9,24.4,42.7,52,0


In [4]:
df['week_start_date'] = pd.to_datetime(df['week_start_date'])

In [5]:
def encode_cyclic_f(data, col, max_val):
    data[col + '_sin'] = np.sin(2 * np.pi * data[col] / max_val)
    data[col + '_cos'] = np.cos(2 * np.pi * data[col] / max_val)

    return data

df['month'] = df['week_start_date'].dt.month
df['dayofweek'] = df['week_start_date'].dt.dayofweek
df['dayofyear'] = df['week_start_date'].dt.dayofyear
df['quarter'] = df['week_start_date'].dt.quarter
df['season'] = df['month'] % 12 // 3 + 1
df['is_weekend'] = df['dayofweek'].apply(lambda x: 1 if x > 4 else 0)
df['is_weekday'] = df['dayofweek'].apply(lambda x: 1 if x < 4 else 0)

df = encode_cyclic_f(df, 'month', 12)
df = encode_cyclic_f(df, 'dayofweek', 6)
df = encode_cyclic_f(df, 'dayofyear', 358)
df = encode_cyclic_f(df, 'quarter', 4)
df = encode_cyclic_f(df, 'season', 4)

In [98]:
df.head()

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,month_sin,month_cos,dayofweek_sin,dayofweek_cos,dayofyear_sin,dayofyear_cos,quarter_sin,quarter_cos,season_sin,season_cos
0,sj,2004,2,2004-01-08,-0.07310,0.01720,0.214729,0.222443,0.00,297.648571,...,5.000000e-01,8.660254e-01,1.224647e-16,-1.0,0.139946,0.990159,1.000000e+00,6.123234e-17,1.000000e+00,6.123234e-17
1,sj,2000,37,2000-09-16,0.05770,0.15070,0.297186,0.254343,30.52,300.794286,...,-1.000000e+00,-1.836970e-16,-8.660254e-01,0.5,-0.988893,-0.148629,-1.000000e+00,-1.836970e-16,-2.449294e-16,1.000000e+00
2,sj,1992,25,1992-06-17,0.10445,0.15315,0.181157,0.192086,52.07,298.931429,...,1.224647e-16,-1.000000e+00,8.660254e-01,-0.5,0.174608,-0.984638,1.224647e-16,-1.000000e+00,-1.000000e+00,-1.836970e-16
3,sj,1991,40,1991-10-01,0.12910,0.16672,0.177000,0.195557,31.68,299.650000,...,-8.660254e-01,5.000000e-01,8.660254e-01,0.5,-0.995345,0.096380,-2.449294e-16,1.000000e+00,-2.449294e-16,1.000000e+00
4,sj,1992,34,1992-08-19,0.11460,0.13680,0.185586,0.192571,68.22,299.491429,...,-8.660254e-01,-5.000000e-01,8.660254e-01,-0.5,-0.801735,-0.597680,-1.000000e+00,-1.836970e-16,-1.000000e+00,-1.836970e-16


In [91]:
!pip install tsfresh

  Using cached tsfresh-0.20.0-py2.py3-none-any.whl (98 kB)
  Using cached distributed-2022.12.1-py3-none-any.whl (930 kB)
  Using cached stumpy-1.11.1-py3-none-any.whl (136 kB)
     -------------------------------------- 925.3/925.3 kB 3.5 MB/s eta 0:00:00
  Using cached tblib-1.7.0-py2.py3-none-any.whl (12 kB)
  Using cached msgpack-1.0.4-cp310-cp310-win_amd64.whl (61 kB)
  Using cached zict-2.2.0-py2.py3-none-any.whl (23 kB)
     -------------------------------------- 923.4/923.4 kB 5.3 MB/s eta 0:00:00
     -------------------------------------- 923.8/923.8 kB 4.5 MB/s eta 0:00:00
     -------------------------------------- 918.2/918.2 kB 5.8 MB/s eta 0:00:00
     -------------------------------------- 908.6/908.6 kB 6.4 MB/s eta 0:00:00
     -------------------------------------- 906.6/906.6 kB 5.7 MB/s eta 0:00:00
     -------------------------------------- 903.8/903.8 kB 5.2 MB/s eta 0:00:00
     -------------------------------------- 902.4/902.4 kB 5.7 MB/s eta 0:00:00
     ----

In [99]:
from tsfresh.feature_extraction import feature_calculators as fc


df['abs_energy'] = fc.abs_energy(df['total_cases'])
df['count_above_mean'] = fc.count_above_mean(df['total_cases'])
df['count_below_mean'] = fc.count_below_mean(df['total_cases'])
df['mean_abs_change'] = fc.mean_abs_change(df['total_cases'])
df['mean_change'] = fc.mean_change(df['total_cases'])
df['sum_values'] = fc.sum_values(df['total_cases'])

In [100]:
df.head()

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,quarter_sin,quarter_cos,season_sin,season_cos,abs_energy,count_above_mean,count_below_mean,mean_abs_change,mean_change,sum_values
0,sj,2004,2,2004-01-08,-0.07310,0.01720,0.214729,0.222443,0.00,297.648571,...,1.000000e+00,6.123234e-17,1.000000e+00,6.123234e-17,3651893,420,1036,30.076976,-0.008247,35927
1,sj,2000,37,2000-09-16,0.05770,0.15070,0.297186,0.254343,30.52,300.794286,...,-1.000000e+00,-1.836970e-16,-2.449294e-16,1.000000e+00,3651893,420,1036,30.076976,-0.008247,35927
2,sj,1992,25,1992-06-17,0.10445,0.15315,0.181157,0.192086,52.07,298.931429,...,1.224647e-16,-1.000000e+00,-1.000000e+00,-1.836970e-16,3651893,420,1036,30.076976,-0.008247,35927
3,sj,1991,40,1991-10-01,0.12910,0.16672,0.177000,0.195557,31.68,299.650000,...,-2.449294e-16,1.000000e+00,-2.449294e-16,1.000000e+00,3651893,420,1036,30.076976,-0.008247,35927
4,sj,1992,34,1992-08-19,0.11460,0.13680,0.185586,0.192571,68.22,299.491429,...,-1.000000e+00,-1.836970e-16,-1.000000e+00,-1.836970e-16,3651893,420,1036,30.076976,-0.008247,35927


In [6]:
df.to_csv('train_folds_updated.csv', index=False)

In [7]:
test = pd.read_csv('dengue_features_test.csv')
test.head()

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,sj,2008,18,2008-04-29,-0.0189,-0.018900,0.102729,0.091200,78.60,298.492857,...,25.37,78.781429,78.60,15.918571,3.128571,26.528571,7.057143,33.3,21.7,75.2
1,sj,2008,19,2008-05-06,-0.0180,-0.012400,0.082043,0.072314,12.56,298.475714,...,21.83,78.230000,12.56,15.791429,2.571429,26.071429,5.557143,30.0,22.2,34.3
2,sj,2008,20,2008-05-13,-0.0015,NaN,0.151083,0.091529,3.66,299.455714,...,4.12,78.270000,3.66,16.674286,4.428571,27.928571,7.785714,32.8,22.8,3.0
3,sj,2008,21,2008-05-20,NaN,-0.019867,0.124329,0.125686,0.00,299.690000,...,2.20,73.015714,0.00,15.775714,4.342857,28.057143,6.271429,33.3,24.4,0.3
4,sj,2008,22,2008-05-27,0.0568,0.039833,0.062267,0.075914,0.76,299.780000,...,4.36,74.084286,0.76,16.137143,3.542857,27.614286,7.085714,33.3,23.3,84.1


In [8]:
test['week_start_date'] = pd.to_datetime(test['week_start_date'])

In [9]:
def encode_cyclic_f(data, col, max_val):
    data[col + '_sin'] = np.sin(2 * np.pi * data[col] / max_val)
    data[col + '_cos'] = np.cos(2 * np.pi * data[col] / max_val)

    return data

test['month'] = test['week_start_date'].dt.month
test['dayofweek'] = test['week_start_date'].dt.dayofweek
test['dayofyear'] = test['week_start_date'].dt.dayofyear
test['quarter'] = test['week_start_date'].dt.quarter
test['season'] = test['month'] % 12 // 3 + 1
test['is_weekend'] = test['dayofweek'].apply(lambda x: 1 if x > 4 else 0)
test['is_weekday'] = test['dayofweek'].apply(lambda x: 1 if x < 4 else 0)

test = encode_cyclic_f(test, 'month', 12)
test = encode_cyclic_f(test, 'dayofweek', 6)
test = encode_cyclic_f(test, 'dayofyear', 358)
test = encode_cyclic_f(test, 'quarter', 4)
test = encode_cyclic_f(test, 'season', 4)

In [10]:
test.fillna(method='ffill', inplace=True)

In [11]:
test.to_csv('test_updated.csv', index=False)